In [1]:
import requests
import pandas as pd
import json
import numpy as np
import pandas as pd
from s3fs.core import S3FileSystem
import boto3


D:\Utils\Anaconda3\envs\p36\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
stations = [{'name': 'Adelsö A' , 'id': 97280}, {'name': 'Tullinge A' , 'id': 97100}]
df = pd.DataFrame()
for station in stations:
    station_url = "https://opendata-download-metobs.smhi.se/api/version/latest/parameter/14/station/{0}/period/latest-months/data.json".format(station['id'])
    r = requests.get(url = station_url)
    data = r.json()
    station_df = pd.DataFrame(data['value'])
    station_df['date'] = pd.to_datetime(station_df['date'], unit='ms')
    station_df = station_df.set_index('date')
    station_df.index = pd.to_datetime(station_df.index)
    station_df.columns = pd.MultiIndex.from_product([[station['name']], station_df.columns], names=['station', 'data'])
    df = pd.concat([df,station_df], axis=1)
df.to_csv('smhi.csv')

In [ ]:
keys = []
bucket_name = '<your destination s3 path>'
s3boto = boto3.resource('s3')
my_bucket = s3boto.Bucket(bucket_name)
for object_summary in my_bucket.objects.filter(Prefix="norrsken/ewd"):
    keys.append(object_summary.key)

In [2]:
data = []
s3fs = S3FileSystem()
for key in keys:
    array = np.load(s3fs.open('{}/{}'.format(bucket_name, key)))
    data.append(array)
    
smhi = pd.read_csv('smhi.csv', header=[0,1], index_col=0, parse_dates=True)
keys_datetime = [pd.to_datetime(k.split('/')[-1].replace('.npy', '')) for k in keys]
smhi = smhi.resample('1T').interpolate(method='linear')
smhi = smhi[pd.to_datetime('2019-09-08 12'):pd.to_datetime('2019-09-09 12')]
smhi = smhi[smhi.index.isin(keys_datetime)]

NameError: name 'keys' is not defined